<a href="https://colab.research.google.com/github/kaz2ngt/colaboratory/blob/master/10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リストPART2

In [0]:
import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

# 10_1: 低解像度の画像の生成

In [0]:
def drop_resolution(x, scale=3.0):
  size = (x.shape[0], x.shape[1])
  small_size = (int(size[0]/scale), int(size[1]/scale))
  img = array_to_img(x)
  small_img = img.resize(small_size, 3)
  return img_to_array(small_img.resize(img.size, 3))

# 10_2: ジェネレータの定義

In [0]:
def data_generator(data_dir, mode, scale=2.0, target_size=(200, 200), batch_size=32, shuffle=True):
  for imgs in ImageDataGenerator().flow_from_directory(
      directory=data_dir,
      classes=[mode],
      class_mode=None,
      color_mode='rgb',
      target_size=target_size,
      batch_size=batch_size,
      shuffle=shuffle
  ):
      x = np.array([
          drop_resolution(img, scale) for img in imgs
      ])
      yield x/255., imgs/255.

# 10_3: data_generatorの利用

In [0]:
# dataの解凍
#!unzip data.zip

In [6]:
DATA_DIR = 'data/chap10/'
N_TRAIN_DATA = 1000
N_TEST_DATA = 100
BATCH_SIZE = 32

train_data_generator = data_generator(DATA_DIR, 'train', batch_size=BATCH_SIZE)
test_x, test_y = next(
    data_generator(
        DATA_DIR,
        'test',
        batch_size=N_TEST_DATA,
        shuffle=False
    )
)

Found 100 images belonging to 1 classes.


# 10_4: SRCNNの定義

In [7]:
model = Sequential()
model.add(Conv2D(
    filters=64,
    kernel_size=9,
    padding='same',
    activation='relu',
    input_shape=(None, None, 3)
))
model.add(Conv2D(
    filters=32,
    kernel_size=1,
    padding='same',
    activation='relu'
))
model.add(Conv2D(
    filters=3,
    kernel_size=5,
    padding='same'
))

model.summary()

W0703 06:59:34.467530 140378264364928 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, None, None, 64)    15616     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 32)    2080      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 3)     2403      
Total params: 20,099
Trainable params: 20,099
Non-trainable params: 0
_________________________________________________________________


# 10_5: ピーク信号対雑音比の定義

In [0]:
def psnr(y_true, y_pred):
  return -10*K.log(
      K.mean(K.flatten((y_true - y_pred))**2)
  )/np.log(10)

# 10_6: PSNRをmetricsに指定して学習を実行

In [9]:
model.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=[psnr]
)

model.fit_generator(
    train_data_generator,
    validation_data=(test_x, test_y),
    steps_per_epoch=N_TRAIN_DATA//BATCH_SIZE,
    epochs=50
)

# テストデータに対して適用
pred = model.predict(test_x)

Found 1000 images belonging to 1 classes.
Epoch 1/50
31/31 [==============================] - 8s 255ms/step - loss: 0.0420 - psnr: 15.4117 - val_loss: 0.0187 - val_psnr: 17.4070
Epoch 2/50
31/31 [==============================] - 4s 144ms/step - loss: 0.0134 - psnr: 18.7634 - val_loss: 0.0138 - val_psnr: 18.7664
Epoch 3/50
31/31 [==============================] - 4s 144ms/step - loss: 0.0099 - psnr: 20.0894 - val_loss: 0.0108 - val_psnr: 19.8987
Epoch 4/50
31/31 [==============================] - 4s 141ms/step - loss: 0.0074 - psnr: 21.3234 - val_loss: 0.0091 - val_psnr: 20.7208
Epoch 5/50
31/31 [==============================] - 4s 142ms/step - loss: 0.0072 - psnr: 21.4837 - val_loss: 0.0084 - val_psnr: 21.0373
Epoch 6/50
31/31 [==============================] - 4s 142ms/step - loss: 0.0061 - psnr: 22.2088 - val_loss: 0.0077 - val_psnr: 21.4466
Epoch 7/50
31/31 [==============================] - 4s 141ms/step - loss: 0.0055 - psnr: 22.6188 - val_loss: 0.0071 - val_psnr: 21.8325
Epoch 

# 10_7: モデルの構築

In [0]:
from tensorflow.python.keras.layers import Add, Conv2DTranspose

# 入力は任意のサイズで、3チャンネルの画像
inputs = Input((None, None, 3), dtype='float')

# Encoder
conv1 = Conv2D(64, 3, padding='same')(inputs)
conv1 = Conv2D(64, 3, padding='same')(conv1)

conv2 = Conv2D(64, 3, strides=2, padding='same')(conv1)
conv2 = Conv2D(64, 3, padding='same')(conv2)

conv3 = Conv2D(64, 3, strides=2, padding='same')(conv2)
conv3 = Conv2D(64, 3, padding='same')(conv3)

# Decoder
deconv3 = Conv2DTranspose(64, 3, padding='same')(conv3)
deconv3 = Conv2DTranspose(64, 3, strides=2, padding='same')(deconv3)

# Add() レイヤーを使ってスキップコネクションを表現
merge2 = Add()([deconv3, conv2])
deconv2 = Conv2DTranspose(64, 3, padding='same')(merge2)
deconv2 = Conv2DTranspose(64, 3, strides=2, padding='same')(deconv2)
merge1 = Add()([deconv2, conv1])
deconv1 = Conv2DTranspose(64, 3, padding='same')(merge1)
deconv1 = Conv2DTranspose(3, 3, padding='same')(deconv1)

output = Add()([deconv1, inputs])

model = Model(inputs, output)